In [11]:
#8:
import pandas as pd
import numpy as np
sales = pd.read_csv('Ejercicio 8 Guia 1 sales.csv')
products = pd.read_csv('Ejercicio 8 Guia 1 products.csv')
sales.head()

,fecha,productid,importe
0,21/02/20,14,1
1,22/03/20,4,4
2,13/01/20,13,6
3,06/02/20,9,9
4,24/02/20,9,9


In [12]:
products.head()

,productid,title,category,description
0,0,A,kids,fine product
1,1,B,men,nice product
2,2,C,kids,shitty product
3,3,D,men,bad product
4,4,E,men,awesome product


In [13]:
#A: Indicar los titulos de la categoria "men" para los cuales el importe de la venta, supera el promedio mensual
#de ventas de los productos de la misma categoria. Usar transform
dfA = sales
dfA = dfA.set_index('productid')
dfA[['category','title']] = products.set_index('productid')[['category','title']]
dfA = dfA.reset_index()
dfA.head()

,productid,fecha,importe,category,title
0,14,21/02/20,1,women,O
1,4,22/03/20,4,men,E
2,13,13/01/20,6,kids,N
3,9,06/02/20,9,women,J
4,9,24/02/20,9,women,J


In [14]:
dfA = dfA.loc[dfA['category'] == 'men']
dfA.head()

,productid,fecha,importe,category,title
1,4,22/03/20,4,men,E
7,4,14/03/20,4,men,E
11,8,15/02/20,2,men,I
17,6,01/02/20,7,men,G
19,8,07/03/20,2,men,I


In [15]:
dfA['mes'] = pd.to_datetime(dfA['fecha']).dt.month
dfA.head()

,productid,fecha,importe,category,title,mes
1,4,22/03/20,4,men,E,3
7,4,14/03/20,4,men,E,3
11,8,15/02/20,2,men,I,2
17,6,01/02/20,7,men,G,1
19,8,07/03/20,2,men,I,7


In [16]:
superaPromedioMes = dfA.groupby('mes')['importe'].transform(lambda x: x > x.mean())
superaPromedioMes.head()

1     False
7     False
11    False
17     True
19    False
Name: importe, dtype: bool

In [17]:
dfA['supera promedio mes'] = superaPromedioMes
dfA.head()

,productid,fecha,importe,category,title,mes,supera promedio mes
1,4,22/03/20,4,men,E,3,False
7,4,14/03/20,4,men,E,3,False
11,8,15/02/20,2,men,I,2,False
17,6,01/02/20,7,men,G,1,True
19,8,07/03/20,2,men,I,7,False


In [18]:
dfA.drop(['productid','fecha','importe','category','mes'], axis = 1).loc[dfA['supera promedio mes']]['title'].unique()

array(['G', 'F', 'H', 'D', 'B'], dtype=object)

In [21]:
#B: Indicar el top-10 de productos que se vendieron mas dias consecutivos
sales['fecha'] = pd.to_datetime(sales['fecha'])
sales.dtypes

fecha        datetime64[ns]
productid             int64
importe               int64
dtype: object

In [24]:
sales = sales.sort_values(by = 'fecha')
sales.head()

,fecha,productid,importe
73,2020-01-01,0,10
17,2020-01-02,6,7
148,2020-01-03,14,1
66,2020-01-04,3,4
2,2020-01-13,13,6


In [38]:
def cantidadDeDiasConsecutivos(df):
    df['shifted'] = df['fecha'].shift()
    df['venta consecutiva'] = (np.timedelta64(1,"D") <= df['fecha'] - df['shifted']) & (df['fecha'] - df['shifted'] < np.timedelta64(2,"D"))
    return df['venta consecutiva'].sum()

ventasConsecutivas = sales.groupby('productid').apply(cantidadDeDiasConsecutivos)
ventasConsecutivas.nlargest(10)

productid
0     2
2     2
5     2
8     1
9     1
12    1
14    1
1     0
3     0
4     0
dtype: int64